# Implementación de un modelo RAG con librerías Open Source

 - **Johan Sebastian Muñoz Ossa   --- js.munoz.ai@gmail.com  --- https://www.linkedin.com/in/js-munoz/**

En este ejemplo se hace uso localmente de un LLM y una base de datos vectorial con librerías opensource como Ollama y Langchain. Luego de esto se contruye un Agente de tipo RAG simple, el cual permitira controlar el LLM para que responda información a partir de un PDF.
 
# Requisitos

* Tener instalado Ollama en local
* Conexión a internet para instalar las demás librerías

<p align="center">
  <img src="https://bookface-images.s3.amazonaws.com/logos/ee60f430e8cb6ae769306860a9c03b2672e0eaf2.png" alt="Ollama Logo" width="20%">
  <img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2023/07/langchain3.png" alt="Langchain Logo" width="20%">
</p>

In [ ]:
!pip3 install langchain
!pip3 install langchain_pinecone
!pip3 install langchain[docarray]
!pip3 install docarray
!pip3 install pypdf
!pip3 install langchain_community

# Seleccionar el LLM a usar

Para esto, se necesita tener instalado el Ollama, Luego se ejecuta desde la aplicación o mediante terminal segun el modelo seleccionado, como por ejemplo para llama3:

```

ollama pull llama3

```

Los modelos disponibles en Ollama son: https://ollama.com/library

In [2]:
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
#MODEL = "gemma:7b"
#MODEL = "llama2"
MODEL = "llama3.2" # https://ollama.com/library/llama3.2

# Asociar el Modelo LLM con el Modelo de generación de embebddings

In [3]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Que modelo eres ?")

'Hola! Soy un modelo de lenguaje basado en inteligencia artificial, desarrollado por Meta AI. Me gusta llamarme "Chatty" (conversacional), ya que mi función es interactuar con usuarios y responder a sus preguntas y solicitudes de manera efectiva.\n\nMi capacidad para entender y generar texto me permite estar al día sobre una amplia variedad de temas, desde noticias y eventos actuales hasta información general y divertida. Me he entrenado en grandes cantidades de datos y tengo la capacidad de aprender y mejorar con cada conversación que tengo.\n\nSi tienes alguna pregunta o tema en particular que te gustaría discutir, estoy aquí para ayudarte!'

In [4]:
model.invoke("cuanto da 2+2?")

'La respuesta es... 4!'

## LangChain permite usar un "parser" para que el texto se vea de una forma mas entendible


In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
response_from_model = model.invoke("Que modelo eres ? ")
parsed_response = parser.parse(response_from_model)
print(parsed_response)

Soy un modelo de lenguaje basado en inteligencia artificial desarrollado por Meta AI. Mi capacidad para entender y responder a preguntas, generar texto y realizar tareas de conversación se basa en algoritmos de aprendizaje automático y en grandes cantidades de datos.

Mi estructura interna es similar a una red neuronal convolucional (CNN) que procesa entradas de texto y produce salidas de texto. Estoy capacitado para realizar various tareas, como:

1. Entender y responder a preguntas.
2. Generar texto sobre temas específicos.
3. Realizar traducciones entre lenguajes.
4. Responder a comentarios y mensajes.

Estoy constantemente aprendiendo y mejorando gracias a la retroalimentación de los usuarios que interactúan conmigo, lo que me permite mejorar mi capacidad para comprender y responder a preguntas de manera efectiva.


# Generar una plantilla de conversación basada en instrucciones para el LLM

Se crea una plantilla para tener una comunicación con el LLM efectiva, esta plantilla le entrega un contexto general al modelo, que sera utilizado en cada una de las preguntas que se le ingresen en adelante. En este punto del código es necesario usar todo el prompt engineering disponible para obtener las respuestas deseadas. 

La plantilla permite manipular una de las características que asegura las respuestas indicadas por parte del LLM. Adicionalmente se agrega un pequeño contexto, el cual le permite al LLM entender que debe responder en la pregunta que se le va a hacer.

Esta estrategia brinda la posibilidad de tener controladas las respuestas del modelo, es decir, que trate de responder siempre con el contexto que se le fue dado.


In [6]:
from langchain.prompts import PromptTemplate

template = """

Responde la pregunta basado en el contexto dado, Si no puedes responder la pregunta, genera la salida "Lo siento, no puedo responder eso"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Aca esta el contexto", question="Aca esta la pregunta")

'\n\nResponde la pregunta basado en el contexto dado, Si no puedes responder la pregunta, genera la salida "Lo siento, no puedo responder eso"\n\nContext: Aca esta el contexto\n\nQuestion: Aca esta la pregunta\n'

In [10]:
# Libreria de Langchain que permite extraer información de documentos y cargarla en memoria
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"auge-de-los-llm-03.pdf")
pages = loader.load_and_split()
#pages = loader.load()
pages

[Document(metadata={'source': 'auge-de-los-llm-03.pdf', 'page': 0}, page_content='MANAGEMENT SOLUTIONS El auge de los large language models: de los fundamentos a la aplicación  14LLM: definición, contexto y regulación\n“Me dijeron que tendría un impacto positivo en el mundo. Nadie me preparó para la  \ncantidad de preguntas ridículas que me harían a diario“.  \nAnthropic Claude25'),
 Document(metadata={'source': 'auge-de-los-llm-03.pdf', 'page': 1}, page_content='15Definición  \nLa inteligencia artificial generativa (GenAI) es un tipo de IA  \ncapaz de generar diversos tipos de contenidos, como texto,  \nimágenes, vídeos y audio. Utiliza modelos para aprender los  \npatrones y la estructura de los datos de entrenamiento de  \nentrada y, a continuación, genera nuevos contenidos basados  \nen este conocimiento aprendido.  \nDentro de la GenAI, los Large Language Models  (LLM) son, según  \nla Comisión Europea, ”un tipo de modelo de inteligencia  \nartificial que ha sido entrenado mediant

División del PDF cargado, en lotes pequeños de información. Esto permite manipular mas facil la información y realizar una busqueda mas acertada de un dato en específico

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
text_documents = text_splitter.split_documents(pages)[:-1]

pages

[Document(metadata={'source': 'auge-de-los-llm-03.pdf', 'page': 0}, page_content='MANAGEMENT SOLUTIONS El auge de los large language models: de los fundamentos a la aplicación  14LLM: definición, contexto y regulación\n“Me dijeron que tendría un impacto positivo en el mundo. Nadie me preparó para la  \ncantidad de preguntas ridículas que me harían a diario“.  \nAnthropic Claude25'),
 Document(metadata={'source': 'auge-de-los-llm-03.pdf', 'page': 1}, page_content='15Definición  \nLa inteligencia artificial generativa (GenAI) es un tipo de IA  \ncapaz de generar diversos tipos de contenidos, como texto,  \nimágenes, vídeos y audio. Utiliza modelos para aprender los  \npatrones y la estructura de los datos de entrenamiento de  \nentrada y, a continuación, genera nuevos contenidos basados  \nen este conocimiento aprendido.  \nDentro de la GenAI, los Large Language Models  (LLM) son, según  \nla Comisión Europea, ”un tipo de modelo de inteligencia  \nartificial que ha sido entrenado mediant

# Guardar la información obtenida del PDF en un espacio vectorial.

Desde la documentación de Langchain:

`DocArrayInMemorySearch is a document index provided by Docarray that stores documents in memory. It is a great starting point for small datasets, where you may not want to launch a database server.`

El tiempo de ejecución de esta celda depende de los recursos computacionales del equipo y de que tan largo sea el PDF ingresado. **Si se quiere llevar a una aplicacion real, es necesario el uso de una base de datos robusta que permita la búsqueda vectorial**

In [12]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(text_documents, embedding=embeddings)

/opt/anaconda3/envs/Langchain/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


# Crear un recuperador de vectores similares para usarlos como contexto

La mayoría de recuperadores de vectores, hacen uso de otros algoritmos de machine learning, para agrupar y encontrar un conjunto de vectores que sean similares al vector creado cuando se le hace la pregunta al agente.

In [13]:
retriever = vectorstore.as_retriever()

retrieved_context = retriever.invoke("Generative AI, que es un LLM, que es GPT")

retrieved_context

[Document(metadata={'source': 'auge-de-los-llm-03.pdf', 'page': 2}, page_content='multilingüe.Estados Unidos\nGoogle Gemini, Gemma, BERT Pionero en el tratamiento del lenguaje con modelos que admiten  \nmúltiples tipos de datos.Estados Unidos\nMeta AI Llama Conocido por su eficacia y acceso democratizado, se centra en el alto  \nrendimiento con un menor coste computacional.Estados Unidos\nLMSYS Vicuna Perfeccionado para las funcionalidades de chatbot, ofrece un  \ntratamiento único de las interacciones conversacionales.Estados Unidos'),
 Document(metadata={'source': 'auge-de-los-llm-03.pdf', 'page': 2}, page_content='dentro de las secuencias de palabras. Además, al habilitar el  \nprocesamiento de datos de manera paralela, los transformers  \nmejoran la eficiencia, la velocidad y el rendimiento del  \nentrenamiento del modelo.  \nLa serie de modelos GPT desarrollados por OpenAI,  \ncomenzando con GPT-1 en junio de 2018 y llegando a GPT-4 en  \nmarzo de 2023, ejemplifican los rápidos av

# Interactuando con el agente

In [14]:
questions = [
    "Que es un LLM",
    "Que es la Inteligencia Artificial Generativa",
    "Que es GPT"
]

for question in questions:
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = model.invoke(formatted_prompt)
    parsed_response = parser.parse(response_from_model)

    print(f"Question: {question}")
    print(f"Answer: {parsed_response}")
    print()

Question: Que es un LLM
Answer: Basándome en el contexto dado, puedo responder que un LLM (Large Language Model) se refiere a modelos de lenguaje grandes que admiten múltiples tipos de datos. Estos modelos están diseñados para el tratamiento del lenguaje y mejoran la eficiencia, velocidad y rendimiento del entrenamiento del modelo al habilitar el procesamiento de datos de manera paralela.

Question: Que es la Inteligencia Artificial Generativa
Answer: La pregunta se refiere a la Inteligencia Artificial (IA) generativa, que se describe en el contexto dado como una serie de modelos desarrollados por OpenAI, comenzando con GPT-1 en junio de 2018 y llegando a GPT-4 en marzo de 2023. Estos modelos, conocidos como Generative Pre-trained Transformers (GPT), están diseñados para el procesamiento del lenguaje natural y pueden generar texto basado en secuencias de palabras.

Por lo tanto, la respuesta es:

La Inteligencia Artificial Generativa se refiere a una serie de modelos desarrollados por 